</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

# Unlimited LLM endpoints
endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]
get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) 

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True) # Scale up batch size


input_dict = {"name": "Dilireba"}
node.get_response = get_groq_response # fast sequential inference 
output_dict = node(input_dict) # use node as a function
print("Output dict: ", output_dict)

Processing LLM queries: 100%|██████████| 20/20 [00:11<00:00,  1.74it/s]


 :: Total time elapsed: 11.51s, 0 errors


Processing LLM queries: 100%|██████████| 22/22 [00:28<00:00,  1.32s/it]


 :: Total time elapsed: 28.99s, 0 errors


Processing LLM queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Compiled solutions:        17
  ⏱️ Time breakdown:
     :: Query time: 23.22s
     :: Evolution time: 11.55s
     :: Evaluation time: 58.45s
     :: Total time: 93.22s


📊 Code 0: Fitness: 66.7%


📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- JsonDecodeError : 
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)AstLiteralError : 
malformed node or string on line 1: <ast.Name object at 0x2bac745e0>
Failed to parse LLM response -- JsonDecodeError : 
Expecting property name enclosed in double quotes: line 1 column 2 (char 1)AstLiteralError : 
malformed node or string on line 1: <ast.Name object at 0x28f39c610>


📊 Code 2: Fitness: 8.3%
-------------------------------------------------------------

In [3]:
output_dict

{'age': 32}

</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [2]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=20, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Processing queries: 100%|██████████| 20/20 [00:16<00:00,  1.20it/s]


 :: Total time elapsed: 16.71s, 0 errors


Processing queries: 100%|██████████| 34/34 [00:28<00:00,  1.19it/s]


 :: Total time elapsed: 28.46s, 0 errors


Processing queries: 0it [00:00, ?it/s]


 :: Total time elapsed: 0.00s, 0 errors
🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Batch size:        17
  ⏱️ Time taken: 85.64 seconds


📊 Code 0: Fitness: 16.7%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not aligned with expected output, Expected: {'age': 32} Predicted: {'age': 87}, Error message: 
Value mismatch for key age: 87 != 32


Input: {'name': 'ChengXiao'}, prediction is not aligned with expected output, Expected: {'age': 26} Predicted: {'age': 33}, Error message: 
Value mismatch for key age: 33 != 26




📊 Code 1: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- No JSON structure found in the provided text.
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstL


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [2]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=20) # Batch_size of 100 gives no slow-down

for plan_dict in plan_dicts:
    visualize_plan_dict(plan_dict, plan.meta_prompt.task)
    break

# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

is_sucess, err_msg = plan.spawn_test_cases(main_test_cases) #  pinned test cases generation

 :: Evolving 20 plans in parallel...


Processing LLM queries: 100%|██████████| 20/20 [00:15<00:00,  1.28it/s]


 :: Total time elapsed: 15.68s, 0 errors
 :: Pseudo-code generated for each plan


Processing LLM queries: 100%|██████████| 20/20 [00:17<00:00,  1.12it/s]


 :: Total time elapsed: 17.92s, 0 errors
 :: Plan_dict generated for each plan


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 195.180958ms
Processing LLM queries: 100%|██████████| 20/20 [00:24<00:00,  1.20s/it]

 :: Total time elapsed: 24.01s, 0 errors
Spawned 3 test cases for all sub-nodes


In [4]:
plan.evolve_sub_nodes() # Completely stuck in the first call, debugging ... 

self = plan 
for i, node_dict in enumerate(self.plan_dict["nodes"]):
    meta_prompt = MetaPrompt(
        task=node_dict["task"],
        func_name=node_dict["name"],
        inputs=node_dict["inputs"],
        outputs=node_dict["outputs"],
        input_types=node_dict["input_types"],
        output_types=node_dict["output_types"],
        mode=PromptMode((node_dict.get("mode", "code")).lower())
    )
    test_cases = self.test_cases_dict[node_dict["name"]]
    node = EvolNode(meta_prompt, None, None, get_response=self.get_response, test_cases=test_cases)
    print(f"🎲 :: Evolving {node.meta_prompt.func_name} ... ({i}/{len(self.plan_dict['nodes'])})")
    node.evolve("i1", replace=True, max_tries=2, num_runs=2, batch_size=20) # It's funny how 30+ sec could elapse before llm inference ... (collecting prompts ?? wtf is taking so long ??)
    self.nodes.append(node)
    break

🎲 :: Evolving search_google ...


Processing LLM queries:   5%|▌         | 1/20 [00:00<00:14,  1.30it/s]

Error in completion: 'NoneType' object has no attribute 'choices'
Error in completion: 'NoneType' object has no attribute 'choices'


Processing LLM queries:  15%|█▌        | 3/20 [00:01<00:05,  3.16it/s]

Error in completion: 'NoneType' object has no attribute 'choices'


Processing LLM queries: 100%|██████████| 20/20 [00:14<00:00,  1.39it/s]


 :: Total time elapsed: 14.40s, 3 errors
